In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [22]:
def h(x1,x2):
    return (np.cos(x1) * np.sin(x2)) - (x1/(np.power((x2),2)+1))

In [23]:
def kromosom_awal(populasi, panjang_kromosom):
    kromosom = []
    for i in range(populasi):
        hasil = ""
        for j in range(panjang_kromosom*2):
            hasil+= str(np.random.randint(2))
        kromosom.append([hasil])  
    return kromosom  


In [24]:
#g = kromosom
#N = Max generasi
#ra,rb = batas atas, batas bawah

def DivSum(N):
    hasil = 0
    for i in range(1,N+1):
        hasil += 1.0/(np.power(2,int(i)))
    return hasil

def SumG(g):
    hasil = 0
    counter = 1
    for i in g:
        hasil += int(i) * (1.0/ np.power(2,counter)) 
        counter += 1
    return hasil

def X(ra1, rb1, ra2, rb2, N, g1, g2):
    return [rb1 + (((ra1 - rb1)/DivSum(N)) * SumG(g1)), rb2 + (((ra2 - rb2)/DivSum(N)) * SumG(g2))] 


In [25]:

def calcx(Kromosom,panjang_kromosom):
    x = []
    for i in Kromosom:
        x.append(X(x1_constraint[1], x1_constraint[0], x2_constraint[1],  x2_constraint[0] , populasi, i[0:panjang_kromosom], i[panjang_kromosom:]))
    return x
    


In [26]:

def fitness_rev(generasi, c):
    return np.power(c,-h(generasi['x1'],generasi['x2']))

In [27]:
def probability_individu(df):
    peluang = []
    for i in df['fitness']:
        peluang.append(i/sum(df['fitness']))
    return peluang

In [28]:
def orangtua_roulette(df):
    roulette_field = []
    ortu = pd.DataFrame([])
    peluang = probability_individu(df)
    sums = 0
    for k in peluang:
        sums += k
        roulette_field.append(sums)
    batas_awal = 0 
    j = 0
    for i in roulette_field:
        rndm = round(np.random.random(),3) + np.random.uniform(0,0.5)
        if (rndm < i):
            ortu = pd.concat([ortu, df.loc[[j]]])
        j+=1
    return ortu

In [29]:
def crossover(sampel):
    p1 = np.random.randint(3, round(length_kromosom/2))
    p2 = np.random.randint(p1, length_kromosom)
    p3 = np.random.randint(p2, length_kromosom*2)

    hasil = []
    bapa = sampel['Kromosom'].values[0]
    ibu = sampel['Kromosom'].values[1]

    anak_1 = bapa[:p1] + ibu[p1:p2] + bapa[p2:p3] + ibu[p3:] 
    anak_2 = ibu[:p1] + bapa[p1:p2] + ibu[p2:p3] + bapa[p3:]
    return anak_1, anak_2

In [30]:
def hasilCrossover_roulette(parents):
    next_gen = []
    jumlah_next_gen = 0
    # print(parents)
    while(jumlah_next_gen <= len(parents['Kromosom'])):
        sampel = parents[['Kromosom']].sample(n=2, replace=True)
        gen = crossover(sampel)
        next_gen.append(gen[0])
        next_gen.append(gen[1])
        jumlah_next_gen =len(next_gen)
    return next_gen

In [31]:

def gencheck(kromosom, Pm):
    hasil = ""
    for i in range(len(kromosom)):
        a = np.random.uniform(0,1)/10
        if(a <= Pm):
            if(kromosom[i] == '0'):
                hasil+='1'
            else:
                hasil+='0'
        else:
            hasil+=kromosom[i]
    return hasil


def mutasi(generasi, Pm):
    mutant = []
    for i in generasi:
        mutant.append(gencheck(i, Pm))
    return mutant
        

In [32]:
def seleksi_rev(prev_gen, next_gens):
    for i in range(len(next_gens['fitness'])):
        minimum_prev = np.argmin(prev_gen['fitness'])
        minimum_next = np.argmin(next_gens['fitness'])
        if(prev_gen['fitness'].loc[minimum_prev] < next_gens['fitness'].loc[minimum_next]):
            prev_gen['Kromosom'].loc[minimum_prev] = next_gens['Kromosom'].loc[minimum_next]
            prev_gen['x1'].loc[minimum_prev] = next_gens['x1'].loc[minimum_next]
            prev_gen['x2'].loc[minimum_prev] = next_gens['x2'].loc[minimum_next]
            prev_gen['fitness'].loc[minimum_prev] = next_gens['fitness'].loc[minimum_next]
            # print("ini", i)
        next_gens = next_gens.drop([minimum_next])
        next_gens = next_gens.reset_index(drop=True)
    return prev_gen

In [33]:
def makegraph(total_fit):
    plt.xlabel("Generasi")
    plt.ylabel("Fitness")
    plt.plot(total_fit)

In [34]:
def XFIT(df,c, len_kromosom):
     x = pd.DataFrame(calcx(df['Kromosom'], len_kromosom), columns = ['x1', 'x2'])
     fit = pd.DataFrame(fitness_rev(x,c), columns=['fitness'])
     df = pd.concat([df, x, fit],  axis=1)
     return df

In [35]:
def ProgramKeturunan(populasi, panjang_kromosom, generasi_akhir, Pm, c):
    gen_1 = pd.DataFrame(kromosom_awal(populasi, panjang_kromosom), columns=['Kromosom'])
    

    gen_1 = XFIT(gen_1, c, panjang_kromosom)

    total_fit = [sum(gen_1['fitness'])]
   

    gen_dummy = gen_1.copy()

    for i in range(1, generasi_akhir):


        gen_dummy_p = orangtua_roulette(gen_dummy)


        gen_lanjut = hasilCrossover_roulette(gen_dummy_p)
        

        gen_lanjut = pd.DataFrame(mutasi(gen_lanjut, Pm), columns = ["Kromosom"])
        
        


        gen_lanjut = XFIT(gen_lanjut, c, panjang_kromosom)
        
        
        gen_lanjut = seleksi_rev(gen_dummy, gen_lanjut)
        total_fit.append(sum(gen_lanjut['fitness']))

        kromosom_dummy = pd.DataFrame(gen_lanjut['Kromosom'], columns=['Kromosom'])


       
        gen_dummy = kromosom_dummy.copy()
        gen_dummy = XFIT(gen_dummy, c, panjang_kromosom)


    #makegraph(total_fit)
    
    return gen_lanjut



In [36]:
x1_constraint = [-1,2]
x2_constraint = [-1,1]
populasi = 30
length_kromosom = 10
Pm = 1/populasi
c = 2
max_generasi = 333

In [37]:
Hasil_generasi = ProgramKeturunan(populasi, length_kromosom, max_generasi, Pm, c)
Hasil_generasi

,Kromosom,x1,x2,fitness
0,11111111101001010100,1.994141,0.164063,4.024945
1,11111111111001000110,1.997070,0.136719,4.046798
2,11111111011001001011,1.991211,0.146484,4.024903
3,11111111001001011000,1.988281,0.171875,4.001600
4,11111111101000111100,1.994141,0.117188,4.042249
5,11111111111001100100,1.997070,0.195313,4.010920
6,11111110101001001001,1.982422,0.142578,3.999383
7,11111111111001000010,1.997070,0.128906,4.049042
8,11111110111000100000,1.985352,0.062500,4.007726
9,11111111001000101101,1.988281,0.087891,4.024001


In [38]:
#INDIVIDU TERBAIK
print("Individu Terbaik")
Hasil_generasi.loc[[np.argmax(Hasil_generasi['fitness'])]]

Individu Terbaik


,Kromosom,x1,x2,fitness
19,11111111111001000001,1.99707,0.126953,4.049505


In [39]:
#Rata-Rata Fitness Generasi Terakhir

# for i in range(10):
#     print(sum(ProgramKeturunan(populasi, length_kromosom, max_generasi, Pm, c)['fitness'])/populasi)
    

In [40]:
# (ProgramKeturunan(populasi, length_kromosom, max_generasi, Pm, c))#.to_excel("Hasil_rev2.xlsx")